In [2]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc
print(datetime.datetime.now())

# date_start=datetime.date(2020,1,1)
date_end=datetime.date(2020,5,2)
date_end

2020-05-27 11:25:28.746553


datetime.date(2020, 5, 2)

In [5]:
list_dcm_click_logs=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/DCM_raw_logs_BL/clicks/*")
list_dcm_click_logs=[x for x in list_dcm_click_logs if "." in x]
print(len(list_dcm_click_logs))
print(set([x.split(".")[1] for x in list_dcm_click_logs]))
list_dcm_click_logs.sort()

df_click_dcm_logs=pd.DataFrame({"file_path":list_dcm_click_logs})
df_click_dcm_logs['utc_date']=df_click_dcm_logs['file_path'].apply(lambda x: x.split("click_utc_")[1][:8])
print(df_click_dcm_logs.shape,df_click_dcm_logs['utc_date'].min(),df_click_dcm_logs['utc_date'].max(),df_click_dcm_logs['utc_date'].nunique())


711
{'tsv'}
(711, 2) 20180523 20200502 711


In [6]:
list_lr_click_returned=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/clicks/*")
print(len(list_lr_click_returned))
print(set([x.split(".")[1] for x in list_lr_click_returned]))
list_lr_click_returned.sort()

df_click_lr_returned=pd.DataFrame({"file_path":list_lr_click_returned})
df_click_lr_returned['utc_date']=df_click_lr_returned['file_path'].apply(lambda x: x.split("account7252_click_")[1][:8])
print(df_click_lr_returned.shape,df_click_lr_returned['utc_date'].min(),df_click_lr_returned['utc_date'].max(),df_click_lr_returned['utc_date'].nunique())


9965
{'tsv'}
(9965, 2) 20180524 20200502 701


In [8]:
df_click_lr_returned[df_click_lr_returned['utc_date']>="20200101"]['utc_date'].nunique()

123

In [9]:
list_common_dates=set(df_click_lr_returned['utc_date'].unique().tolist()).intersection(set(df_click_dcm_logs['utc_date'].unique().tolist()))
list_common_dates=list(list_common_dates)
list_common_dates.sort()


In [10]:
writer_mapping="/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/UID_IDL_mapping/"
try:
    os.stat(writer_mapping)
except:
    os.mkdir(writer_mapping)

In [11]:
df_uid_idl_mapping=pd.DataFrame()
error_file_list_lr_returned=[]
error_file_list_dcm_logs=[]
i_counter=0
for date in list_common_dates:
    i_counter+=1
    print(i_counter,date,datetime.datetime.now())
    
    df_lr_returned=pd.DataFrame()
    df_dcm_logs=pd.DataFrame()
    
    file_list_lr=df_click_lr_returned[df_click_lr_returned['utc_date']==date]['file_path'].tolist()
    file_list_dcm=df_click_dcm_logs[df_click_dcm_logs['utc_date']==date]['file_path'].tolist()
    
    for file in file_list_lr:
        try:
            df=pd.read_csv(file,dtype=str,sep="\t")
            df=df[df['Advertiser ID']=="8095847"]
            df_lr_returned=df_lr_returned.append(df)
        except:
            error_file_list_lr_returned.append(file)
            print("error_file_list_lr_returned: ",file)
    
    for file in file_list_dcm:
        try:
            df=pd.read_csv(file,dtype=str,sep="\t")
            df=df[df['Advertiser ID']=="8095847"]
            df_dcm_logs=df_dcm_logs.append(df)
        except:
            error_file_list_dcm_logs.append(file)
            print("error_file_list_dcm_logs: ",file)
            
        
    merge_col_list=df_dcm_logs.columns.tolist()
    for col in ['User ID', 'Partner1 ID',"Partner2 ID"]:
        try:
            merge_col_list.remove(col)
        except:
            pass
    df_lr_returned=df_lr_returned[['Customer_Link']+merge_col_list]
    df_lr_returned=df_lr_returned.drop_duplicates()

    df_lr_returned=df_lr_returned[df_lr_returned['Customer_Link']!="UNMATCHED"]
    
    df_dcm_logs=df_dcm_logs.drop_duplicates()
    
    df_mapping=pd.merge(df_dcm_logs,df_lr_returned,on=merge_col_list,how="inner")
    df_mapping=df_mapping[['Customer_Link','User ID']]
    df_mapping['file_date']=date
    
    df_uid_idl_mapping=df_uid_idl_mapping.append(df_mapping)
    df_uid_idl_mapping=df_uid_idl_mapping.drop_duplicates(['Customer_Link','User ID'])
    

date_min=df_uid_idl_mapping['file_date'].min()
date_max=df_uid_idl_mapping['file_date'].max()

df_uid_idl_mapping.to_csv(writer_mapping+"BL_GoogleID_IDL_based_on_click_mapping_%s_%s_JL_%s.csv" % (str(date_min),str(date_max),str(datetime.datetime.now().date())),index=False)
gc.collect()

1 20180524 2020-05-27 11:30:29.033463
2 20180525 2020-05-27 11:30:56.058583
3 20180526 2020-05-27 11:31:03.403160
4 20180527 2020-05-27 11:31:13.552447
5 20180528 2020-05-27 11:31:19.776945
6 20180529 2020-05-27 11:31:33.407769
7 20180530 2020-05-27 11:31:41.091066
8 20180531 2020-05-27 11:31:48.661026
9 20180601 2020-05-27 11:31:57.105866
10 20180602 2020-05-27 11:32:02.646042
11 20180603 2020-05-27 11:32:11.982435
12 20180604 2020-05-27 11:32:18.893812
13 20180605 2020-05-27 11:32:28.931917
14 20180606 2020-05-27 11:32:35.564743
15 20180607 2020-05-27 11:32:41.209852
16 20180608 2020-05-27 11:32:47.370683
17 20180609 2020-05-27 11:32:57.401098
18 20180610 2020-05-27 11:33:03.294912
19 20180611 2020-05-27 11:33:14.134567
20 20180612 2020-05-27 11:33:25.445022
21 20180613 2020-05-27 11:33:33.293730
22 20180614 2020-05-27 11:33:41.349188
23 20180615 2020-05-27 11:34:03.457585
24 20180616 2020-05-27 11:34:22.108560
25 20180617 2020-05-27 11:34:36.766057
26 20180618 2020-05-27 11:34:56.31

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


166 20181105 2020-05-27 11:57:32.566601
167 20181106 2020-05-27 11:57:43.279245
168 20181107 2020-05-27 11:57:53.489860
169 20181108 2020-05-27 11:58:04.127575
170 20181109 2020-05-27 11:58:10.747931
171 20181110 2020-05-27 11:58:17.612829
172 20181111 2020-05-27 11:58:26.847546
173 20181112 2020-05-27 11:58:36.447335
174 20181113 2020-05-27 11:58:46.450684
175 20181114 2020-05-27 11:58:54.831894
176 20181115 2020-05-27 11:59:03.725870
177 20181116 2020-05-27 11:59:12.213902
178 20181117 2020-05-27 11:59:20.439966
179 20181118 2020-05-27 11:59:29.404054
180 20181119 2020-05-27 11:59:39.901322
181 20181120 2020-05-27 11:59:50.971511
182 20181121 2020-05-27 12:00:01.864810
183 20181122 2020-05-27 12:00:13.263701
184 20181123 2020-05-27 12:00:24.131592
185 20181124 2020-05-27 12:00:35.737166
186 20181125 2020-05-27 12:00:45.786172
187 20181126 2020-05-27 12:00:55.029002
188 20181127 2020-05-27 12:01:05.045801
189 20181128 2020-05-27 12:01:14.875001
190 20181129 2020-05-27 12:01:24.357796


error_file_list_lr_returned:  /home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/clicks/CL_BigLots_dcm_account7252_click_2019032117_20190321_215214_2107187682_BL_LR_20190322_042023_0000.tsv.gz
error_file_list_lr_returned:  /home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/clicks/CL_BigLots_dcm_account7252_click_2019032118_20190321_220115_2107205917_BL_LR_20190322_035626_0000.tsv.gz
error_file_list_lr_returned:  /home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/clicks/CL_BigLots_dcm_account7252_click_2019032119_20190321_215806_2107226166_BL_LR_20190322_044119_0000.tsv.gz
error_file_list_lr_returned:  /home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/clicks/CL_BigLots_dcm_account7252_click_2019032120_20190321_223521_2107253015_BL_LR_20190322_044658_0000.tsv.gz
error_file_list_lr_returned:  /home/jian/Projects/Big_Lots/Analy

447 20190822 2020-05-27 12:52:23.162284
448 20190823 2020-05-27 12:52:31.937887
449 20190824 2020-05-27 12:52:40.784251
450 20190825 2020-05-27 12:53:06.801107
451 20190826 2020-05-27 12:53:16.723545
452 20190827 2020-05-27 12:53:35.311442
453 20190828 2020-05-27 12:53:45.700662
454 20190829 2020-05-27 12:53:54.599143
455 20190830 2020-05-27 12:54:17.646295
456 20190831 2020-05-27 12:54:26.564464
457 20190901 2020-05-27 12:54:48.147331
458 20190902 2020-05-27 12:54:59.316068
459 20190903 2020-05-27 12:55:23.371633
460 20190904 2020-05-27 12:55:59.289216
461 20190905 2020-05-27 12:56:27.492143
462 20190906 2020-05-27 12:56:37.503328
463 20190907 2020-05-27 12:56:47.245749
464 20190908 2020-05-27 12:56:57.001171
465 20190909 2020-05-27 12:57:06.497865
466 20190910 2020-05-27 12:57:15.930946
467 20190911 2020-05-27 12:57:25.649465
468 20190912 2020-05-27 12:57:45.231164
469 20190913 2020-05-27 12:58:37.312682
470 20190914 2020-05-27 12:59:02.366429
471 20190915 2020-05-27 12:59:32.942363


652 20200314 2020-05-27 13:39:33.711979
653 20200315 2020-05-27 13:39:45.335797
654 20200316 2020-05-27 13:39:57.048335
655 20200317 2020-05-27 13:40:08.691520
656 20200318 2020-05-27 13:40:20.207449
657 20200319 2020-05-27 13:40:32.540916
658 20200320 2020-05-27 13:40:46.681777
659 20200321 2020-05-27 13:40:59.652787
660 20200322 2020-05-27 13:41:11.983718
661 20200323 2020-05-27 13:41:24.355602
662 20200324 2020-05-27 13:41:35.955287
663 20200325 2020-05-27 13:41:47.606043
664 20200326 2020-05-27 13:41:59.297786
665 20200327 2020-05-27 13:42:11.434866
666 20200328 2020-05-27 13:42:24.449571
667 20200329 2020-05-27 13:42:37.028209
668 20200330 2020-05-27 13:42:49.358573
669 20200331 2020-05-27 13:43:02.167895
670 20200401 2020-05-27 13:43:16.587921
671 20200402 2020-05-27 13:43:28.531029
672 20200403 2020-05-27 13:43:40.392795
673 20200404 2020-05-27 13:43:54.276268
674 20200405 2020-05-27 13:44:10.501616
675 20200406 2020-05-27 13:44:23.164868
676 20200407 2020-05-27 13:44:38.592455


923

In [12]:
print("len(error_file_list_lr_returned)",len(error_file_list_lr_returned))
print("len(error_file_list_dcm_logs)",len(error_file_list_dcm_logs))
df_error_file_list_lr_returned=pd.DataFrame({"file_path":error_file_list_lr_returned})
df_error_file_list_lr_returned.to_csv(writer_mapping+"df_error_file_list_lr_returned.csv",index=False)

len(error_file_list_lr_returned) 24
len(error_file_list_dcm_logs) 0


In [13]:
# looks all right since multiple files exising, and the errro is mostly about the header name
df_click_lr_returned[df_click_lr_returned['utc_date']=="20190321"].shape

(46, 2)

In [14]:
writer_mapping

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/UID_IDL_mapping/'

In [15]:
df_uid_idl_mapping_click=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/UID_IDL_mapping/BL_GoogleID_IDL_based_on_click_mapping_20180524_20200502_JL_2020-05-27.csv",
                               dtype=str)

In [16]:
df_uid_idl_mapping_click.shape

(3775908, 3)

In [17]:
df_uid_idl_mapping_click['file_date'].nunique()

700

In [18]:
df_uid_idl_mapping_click.head(2)

,Customer_Link,User ID,file_date
0,XY1468VqI7e_lJ8CJetp6iKvfSPI_tIdVmm1KnxyhtSmVX4_Y,AMsySZauTlkpN1OLtyjNDsUCPbt-,20180524
1,XY1468Wd5VAtYM2UfvM4wzYXlKg0d3LMt9J4IxIWMs2D3ppIM,AMsySZYrTbbBZf5JfA07dQ2CdEa0,20180524


In [19]:
df_uid_idl_mapping_click['Customer_Link'].nunique()

3625099

In [20]:
df_uid_idl_mapping_click['User ID'].nunique()

3637501